## HackAI
Imports:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf
sns.set()

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_csv("../data/flights_searches.csv")
df.head()

In [ ]:
df[["arrival_date", "return_date"]] = df[["arrival_date", "return_date"]].apply(pd.to_datetime)

In [ ]:
df.head()

### Demand by day between 2 airports
The figure below shows the demand on each day between Barcelona and Athens airports

In [ ]:
plt.figure()
bcn_to_ath = df.loc[(df['origin'] == "ROM") & (df['destination'] == "BER") ]
x = bcn_to_ath['arrival_date']
y1 = bcn_to_ath['volume']
plt.plot(x, y1)

In [ ]:
bcn_to_ath['volume'] = bcn_to_ath['volume'].diff()
bcn_to_ath.head()

## Learning the baseline
To identify the impact of a event, it is important to know the excess demand compared to the baseline demand.

How many uniquie routes are there?

In [ ]:
len(df.drop_duplicates(['origin', 'destination']).index)

### Splitting into training and testing data
Because each route has its own unique patterns, a unique model will have to be constructed for each route.
The training set will use the first 70% of the days on each route and it will be tested on the subsequent 30% of values.

In [ ]:
#bcn_to_ath = bcn_to_ath[(np.abs(stats.zscore(bcn_to_ath[['volume']]) < 0.1)).all(axis=1)]
#bcn_to_ath.head()

In [ ]:
sc = MinMaxScaler(feature_range=(-1,1))
bcn_to_ath['volume'] = sc.fit_transform(bcn_to_ath[['volume']])

In [ ]:
plt.figure()
x = bcn_to_ath['arrival_date']
y1 = bcn_to_ath['volume']
plt.plot(x, y1)

In [ ]:
"Number of entries from ROM to BER: {}, ".format(len(bcn_to_ath.index)) +\
"Training set size: {}, ".format(round(len(bcn_to_ath.index)*0.7)) +\
"Testing set size: {}".format(len(bcn_to_ath.index) - round(len(bcn_to_ath.index)*0.7))

In [ ]:
train, test = train_test_split(bcn_to_ath.sort_values(by='arrival_date'), test_size=0.3, shuffle=False)

In [ ]:
train = train[['volume']].values
test = test[['volume']].values

Inititalise an empty training and testing set
The model is given 20 consecutive values and has to predict the 21st

In [ ]:
lookback_length = 20
X_train = np.empty([train.shape[0]-lookback_length, lookback_length, 1])
Y_train = np.empty([train.shape[0]-lookback_length, 1])

X_test = np.empty([test.shape[0]-lookback_length, lookback_length, 1])
Y_test = np.empty([test.shape[0]-lookback_length, 1])
Y_train.shape

In [ ]:
for i in range(lookback_length, train.shape[0]):
  X_train[i-lookback_length] = train[i-lookback_length:i]
  Y_train[i-lookback_length] = train[i]

In [ ]:
for i in range(lookback_length, test.shape[0]):
  X_test[i-lookback_length] = test[i-lookback_length:i]
  Y_test[i-lookback_length] = test[i]

## Making the data stationary
To prevent the model learning specific volume values, the model is given the change in demand from the previous day.
Without this the model is susceptible to autocorrelation which it will ultimately become a persistence model.

In [ ]:
model = Sequential(
    [
        LSTM(units = lookback_length, return_sequences = True, input_shape = (lookback_length,1)),

        Dropout(0.2),

        LSTM(units=lookback_length, return_sequences=True),

        Dropout(0.2),

        LSTM(units=lookback_length, return_sequences=True),

        Dropout(0.2),

        LSTM(units=lookback_length),

        Dense(units=1)

    ]
)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()